In [4]:
# Import all necessary libraries
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import urllib.parse as urlparse
from urllib.parse import urlencode
import re
import time

Assign the base URL for the census data
base_url = "https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/page.cfm?Lang=E"

# List of all Toronto postal codes 
# So that the file can be run independent from the Toronto_Neighborhoods.csv
postal_codes = [
    'M1B', 'M1C', 'M1E', 'M1G', 'M1H', 'M1J', 'M1K', 'M1L', 'M1M', 'M1N',
    'M1P', 'M1R', 'M1S', 'M1T', 'M1V', 'M1W', 'M1X', 'M2H', 'M2J', 'M2K',
    'M2L', 'M2M', 'M2N', 'M2P', 'M2R', 'M3A', 'M3B', 'M3C', 'M3H', 'M3J',
    'M3K', 'M3L', 'M3M', 'M3N', 'M4A', 'M4B', 'M4C', 'M4E', 'M4G', 'M4H',
    'M4J', 'M4K', 'M4L', 'M4M', 'M4N', 'M4P', 'M4R', 'M4S', 'M4T', 'M4V',
    'M4W', 'M4X', 'M4Y', 'M5A', 'M5B', 'M5C', 'M5E', 'M5G', 'M5H', 'M5J',
    'M5K', 'M5L', 'M5M', 'M5N', 'M5P', 'M5R', 'M5S', 'M5T', 'M5V', 'M5W',
    'M5X', 'M6A', 'M6B', 'M6C', 'M6E', 'M6G', 'M6H', 'M6J', 'M6K', 'M6L',
    'M6M', 'M6N', 'M6P', 'M6R', 'M6S', 'M7A', 'M7R', 'M7Y', 'M8V', 'M8W',
    'M8X', 'M8Y', 'M8Z', 'M9A', 'M9B', 'M9C', 'M9L', 'M9M', 'M9N', 'M9P',
    'M9R', 'M9V', 'M9W'
]

# Creating an empty DataFrame to store the results
df = pd.DataFrame(columns=['Postal Code', 'Population', 'Income', 'Number of Old Housings', 'Number of Apartment Units', 'Number of Health Professionals'])

# Loop through each postal code
for postal_code in postal_codes:
    # Update the URL with the specific postal code parameters
    params = {
        'SearchText': postal_code,
        'DGUIDlist': f'2021A0011{postal_code}'
    }
    url_parts = list(urlparse.urlparse(base_url))
    query = dict(urlparse.parse_qsl(url_parts[4]))
    query.update(params)
    url_parts[4] = urlencode(query)
    query_url = urlparse.urlunparse(url_parts)

    try:
        # Use the urlopen function to open the webpage
        html = urllib.request.urlopen(query_url)

        # Create the BeautifulSoup object
        html_to_parse = BeautifulSoup(html, "html.parser")

        # Find all tables on the page
        tables = html_to_parse.find_all("table")
        print(f"Number of tables found for {postal_code}: {len(tables)}")

        # Extract specific data points from the tables
        population = None
        total_income = None
        old_housings = 0
        apartment_units = 0
        health_professions = 0
        
        # Extract relevant data from the tables
        for table in tables:
            if "Population" in table.text:
                td = table.find(attrs={"title": re.compile(r"Population, 2021 - Counts - Total")})
                if td:
                    population = float(td.text.replace(",", ""))
            if "Total - Income statistics in 2020" in table.text:
                td = table.find(attrs={"title": re.compile(r"Total - Income statistics in 2020 for the population aged 15 years and over in private households - 100% data")})
                if td:
                    total_income = float(td.text.replace(",", ""))
            if any(period in table.text for period in ["1960 or before", "1961 to 1980", "1981 to 1990", "1991 to 2000"]):
                td = table.find(attrs={"title": re.compile(r"1960 or before|1961 to 1980|1981 to 1990|1991 to 2000")})
                if td:
                    old_housings += int(td.text.replace(",", ""))
            if any(apartment_type in table.text for apartment_type in [
                "Apartment or flat in a duplex",
                "Apartment in a building that has fewer than five storeys",
                "Apartment in a building that has five or more storeys"
            ]):
                td = table.find(attrs={"title": re.compile(r"Apartment or flat in a duplex|Apartment in a building that has fewer than five storeys|Apartment in a building that has five or more storeys")})
                if td:
                    apartment_units += int(td.text.replace(",", ""))
            if "Health occupations" in table.text:
                td = table.find(attrs={"title": re.compile(r"Health occupations")})
                if td:
                    health_professions = int(td.text.replace(",", ""))

        # Append the data to the DataFrame if values are found
        if population is not None and total_income is not None and old_housings != 0:
            df.loc[len(df.index)] = [postal_code, population, total_income, old_housings, apartment_units, health_professions]
    except Exception as e:
        print(f"Failed to retrieve data for postal code {postal_code}: {e}")

    # Pause to prevent overloading the server
    time.sleep(1)

# Filter the DataFrame to include only rows with postal codes in Toronto
toronto_df = df[df['Postal Code'].str.startswith('M')]

# Save the data to a CSV file
toronto_df.to_csv("toronto_neighbourhood_population_income_old_housings_apartments_health.csv", index=False)
print("Data has been exported to toronto_neighbourhood_population_income_old_housings_apartments_health.csv")

# Display the DataFrame
print(toronto_df)


Number of tables found for M1B: 1
Number of tables found for M1C: 1
Number of tables found for M1E: 1
Number of tables found for M1G: 1
Number of tables found for M1H: 1
Number of tables found for M1J: 1
Number of tables found for M1K: 1
Number of tables found for M1L: 1
Number of tables found for M1M: 1
Number of tables found for M1N: 1
Number of tables found for M1P: 1
Number of tables found for M1R: 1
Number of tables found for M1S: 1
Number of tables found for M1T: 1
Number of tables found for M1V: 1
Number of tables found for M1W: 1
Number of tables found for M1X: 1
Number of tables found for M2H: 1
Number of tables found for M2J: 1
Number of tables found for M2K: 1
Number of tables found for M2L: 1
Number of tables found for M2M: 1
Number of tables found for M2N: 1
Number of tables found for M2P: 1
Number of tables found for M2R: 1
Number of tables found for M3A: 1
Number of tables found for M3B: 1
Number of tables found for M3C: 1
Number of tables found for M3H: 1
Number of tabl